In [ ]:
# Author: Varun Gujarathi
# Modified: 7th Dec 23 (Varun)
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


#Setup GPU for Pandas:



In [ ]:
# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py


Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 395, done.
remote: Counting objects: 100% (126/126), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 395 (delta 92), reused 53 (delta 51), pack-reused 269
Receiving objects: 100% (395/395), 108.50 KiB | 867.00 KiB/s, done.
Resolving deltas: 100% (194/194), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 1.7 MB/s eta 0:00:00
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pynvml/nvml.py", line 1798, in _LoadNvmlLibrary
    nvmlLib = CDLL("libnvidia-ml.so.1")
  File "/usr/lib/python3.10/ctypes/__init__.py", line 374, in __init__
    self._handle = _dlopen(self._name, mode)
OSError: libnvidia-ml.so.1: cannot open shared object file: No such file or directory

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/content/rapidsai-csp-utils/colab/pip-install.py", line 17, in <module>
    pynvml.n

# Gap Sizing

## Import dependencies

In [1]:
# Author: Varun Gujarathi
# Modified: 14th Nov 23 (Varun)
# %load_ext cudf.pandas
import pandas as pd
import sys
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

link_final = 'https://drive.google.com/file/d/19qI1fZjII8PaHrJzGHTp4uLWlqqDHXAO/view?usp=drive_link'
# to get the id part of the file
id = link_final.split("/")[-2]

downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('md_final_sample.csv')

df = pd.read_csv('md_final_sample.csv', low_memory=False)
df_orig = df.copy()

link_brute_force = 'https://drive.google.com/file/d/1t_MLb3FGlAqo2jifJR8cs5bBpDpvzUBY/view?usp=drive_link'
# to get the id part of the file
id = link_brute_force.split("/")[-2]

downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('md_brute_force.csv')

df_brute_force = pd.read_csv('md_brute_force.csv', low_memory=False)
df_brute_force_orig = df_brute_force.copy()

link_disposition_list = 'https://drive.google.com/file/d/1-8EQQUDPiawqFZQ2U1CZ5XXQnOGzyPXc/view?usp=drive_link'
# to get the id part of the file
id = link_disposition_list.split("/")[-2]

downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('md_disposition_list.csv')

df_disposition_list = pd.read_csv('md_disposition_list.csv', low_memory=False)
df_disposition_list_orig = df_disposition_list.copy()



## Explore Data

In [ ]:
df = df_orig.copy()

In [ ]:
df_orig.columns

Index(['Unnamed: 0', 'case_number', 'name', 'race', 'sex', 'DOB', 'DOB_str',
       'city', 'state', 'zip_code', 'person_id', 'id', 'charge_number',
       'cjis_code', 'statute_code', 'charge_description', 'charge_class',
       'probable_cause', 'offense_date_from', 'offense_date_from_str',
       'offense_date_to', 'offense_date_to_str', 'agency_name', 'officer_id',
       'plea', 'plea_date', 'plea_date_str', 'disposition', 'disposition_date',
       'disposition_date_str', 'converted_disposition', 'jail_life',
       'jail_death', 'jail_start_date', 'jail_start_date_str', 'jail_years',
       'jail_months', 'jail_days', 'jail_hours', 'jail_suspended_years',
       'jail_suspended_months', 'jail_suspended_days', 'jail_suspended_hours',
       'jail_suspend_all_but_years', 'jail_suspend_all_but_months',
       'jail_suspend_all_but_days', 'jail_suspend_all_but_hours',
       'jail_suspended_term', 'expunged', 'jail_cons_conc', 'plea_judge',
       'disposition_judge', 'notes', 'sent

In [ ]:
df = df.loc[:,['case_number', 'race', 'sex', 'city', 'state', 'zip_code', 'person_id',
       'charge_number', 'cjis_code', 'statute_code', 'charge_description',
       'charge_class',
       'disposition', 'disposition_date',
       'jail_years', 'jail_months', 'jail_days',
       'jail_hours', 'jail_suspended_years', 'jail_suspended_months',
       'jail_suspended_days', 'jail_suspended_hours',
       'jail_suspend_all_but_years', 'jail_suspend_all_but_months',
       'jail_suspend_all_but_days', 'jail_suspend_all_but_hours',
       'jail_suspended_term', 'expunged',
       'statute_description',
       'victim_age']]

In [ ]:
df.person_id.nunique()

75000

In [ ]:
df.charge_class.value_counts(dropna=False)

Misdemeanor              513643
NaN                      418302
Felony Circuit Court      92877
Felony District Court     71593
Traffic                   37619
Converted Degree          26592
Other                      3469
Name: charge_class, dtype: int64

In [ ]:
df.disposition = df.disposition.str.lower()

In [ ]:
df.disposition.unique()

array(['pbj unsupervised', 'stet', 'guilty', 'nolle prosequi',
       'dismissed', 'abated by death',
       'probation before judgment - unsupervised', 'not guilty',
       'judgment of acquital', 'remanded - jtp district court',
       'appeal dismissed', 'guilty - prepaid',
       'probation before judgment', 'not criminally responsible',
       'not criminally responsible - released',
       'probation before judgment - 641', 'no verdict', 'pbj supervised',
       'forwarded - circuit court', 'guilty - merged for sentencing',
       'probation before judgment - supervised',
       'incompetent to stand trial', 'judgment of acquittal',
       'guilty-merged', 'appeal withdrawn', 'g', 'vmr', 'extradition',
       'pbj', 'lesser included offense', 'acquitted', 'compromised',
       'consent order', 'probation continued', 'change of venue', 'ctwf',
       'jury trial prayed', 'nolo contendere',
       'transferred to other jurisdiction', 'compromise',
       'guilty - not criminally re

In [ ]:
df['statute_code'].isna().sum()/df['statute_code'].shape[0]*100

6.070638564721952

In [ ]:
# Fill missing statute codes
df.statute_code = df.groupby('charge_description').statute_code.ffill()
df.statute_code = df.groupby('charge_description').statute_code.bfill()

In [ ]:
# Fill missing charge class
df.charge_class = df.groupby('statute_code').charge_class.ffill()
df.charge_class = df.groupby('statute_code').charge_class.bfill()

In [ ]:
# Only take Misdemeanor & Felony convictions which are Guilty
df_guilty = df.loc[df['charge_class'].isin(['Misdemeanor','Felony Circuit Court','Felony District Court']) & df['disposition'].isin(['guilty', 'g', 'guilty - merged for sentencing', 'guilty - prepaid', 'glpp', 'guilty - merged for sentencing', 'guilty-merged', 'nolo contendere', 'nolle contendre', 'probation after conviction','sentenced','probation terminated','guilty verdict - no sentence','mand. sentence w/o parole (handgun statute)','life without parole (murder statute)','nolo contendere','mandatory sentence','conviction comp. case','guilty - domestic rltd']),:]

In [ ]:
df_guilty.person_id.nunique()

23941

In [ ]:
# take only not expunged
df_guilty = df_guilty.loc[df_guilty['expunged'] == 'f' ,:]

In [ ]:
# df_brute_force['charge_label'] = df_brute_force['Charge Label'].copy()
# df_brute_force = df_brute_force.drop(columns=['Charge Label'])

In [ ]:
df_guilty['disposition'].value_counts()

guilty                            108079
guilty - merged for sentencing      8100
guilty-merged                       1004
g                                    511
nolo contendere                        4
nolle contendre                        3
Name: disposition, dtype: int64

In [ ]:
from google.colab.data_table import DataTable
DataTable.max_columns = 52

In [ ]:
df_guilty[~df_guilty['jail_hours'].isna() & df_guilty['jail_hours'] != 0.0].head()

,case_number,race,sex,city,state,zip_code,person_id,charge_number,cjis_code,statute_code,...,jail_suspended_days,jail_suspended_hours,jail_suspend_all_but_years,jail_suspend_all_but_months,jail_suspend_all_but_days,jail_suspend_all_but_hours,jail_suspended_term,expunged,statute_description,victim_age
651507,08K03000938,Black,Female,Oxon Hill,MD,20745,3217290970,1,3-5010,CR.7.104,...,NaN,NaN,0.0,0.0,2.0,0.0,NaN,f,NaN,NaN
1014956,08K14000882,Black,Male,Clinton,MD,207350000,8750353522,1,1-0573,CR.5.601.(a)(1),...,NaN,NaN,0.0,0.0,0.0,24.0,NaN,f,NaN,NaN


In [ ]:
df.columns

Index(['case_number', 'race', 'sex', 'city', 'state', 'zip_code', 'person_id',
       'charge_number', 'cjis_code', 'statute_code', 'charge_description',
       'charge_class', 'disposition', 'disposition_date', 'jail_years',
       'jail_months', 'jail_days', 'jail_hours', 'jail_suspended_years',
       'jail_suspended_months', 'jail_suspended_days', 'jail_suspended_hours',
       'jail_suspend_all_but_years', 'jail_suspend_all_but_months',
       'jail_suspend_all_but_days', 'jail_suspend_all_but_hours',
       'jail_suspended_term', 'expunged', 'statute_description', 'victim_age'],
      dtype='object')

## Eligiblity Calculations

In [ ]:
current_date = pd.to_datetime("01-01-2021", infer_datetime_format=True)

two_half_year_delta = pd.Timedelta("913 days")
three_half_year_delta = pd.Timedelta("1278 days")

three_year_delta = pd.Timedelta("1095 days")
five_year_delta = pd.Timedelta("1826 days")
seven_year_delta = pd.Timedelta("2557 days")
ten_year_delta = pd.Timedelta("3652 days")
fifteen_year_delta = pd.Timedelta("5479 days")

def check_eligibility(row):
  # Calculate jail time

  jail_time = None

  # jail years
  try:
    if not pd.isna(row['jail_years']) and row['jail_years'] != 0.0:
      jail_time = pd.Timedelta(str(row['jail_years'] *  365.25) + " days")
      if not pd.isna(row['jail_suspended_years']) and row['jail_suspended_years'] != 0.0:
        jail_time -= pd.Timedelta(str(row['jail_suspended_years'] *  365.25) + " days")

    # if not pd.isna(row['jail_term_years']) and row['jail_term_years'] != 0.0:
    #   jail_time = pd.Timedelta(str(row['jail_term_years'] *  365.25) + " days")
    #   if not pd.isna(row['suspended_term_years']) and row['suspended_term_years'] != 0.0:
    #     jail_time -= pd.Timedelta(str(row['suspended_term_years'] *  365.25) + " days")
  except:
    print(row['jail_years'])

  # jail months
  if not pd.isna(row['jail_months']) and row['jail_months'] != 0.0:
    if jail_time is None:
      jail_time = pd.Timedelta(str(row['jail_months'] *  30) + " days")
    else:
      jail_time += pd.Timedelta(str(row['jail_months'] *  30) + " days")

    if not pd.isna(row['jail_suspended_months']) and row['jail_suspended_months'] != 0.0:
      jail_time -= pd.Timedelta(str(row['jail_suspended_months'] *  30) + " days")

  # if not pd.isna(row['jail_term_months']) and row['jail_term_months'] != 0.0:
  #   if jail_time is None:
  #     jail_time = pd.Timedelta(str(row['jail_term_months'] *  30) + " days")
  #   else:
  #     jail_time += pd.Timedelta(str(row['jail_term_months'] *  30) + " days")

  #   if not pd.isna(row['suspended_term_months']) and row['suspended_term_months'] != 0.0:
  #     jail_time -= pd.Timedelta(str(row['suspended_term_months'] *  30) + " days")


  # jail days
  if not pd.isna(row['jail_days']) and row['jail_days'] != 0.0:
    if jail_time is None:
      jail_time = pd.Timedelta(str(row['jail_days']) + " days")
    else:
      jail_time += pd.Timedelta(str(row['jail_days']) + " days")

    if not pd.isna(row['jail_suspended_days']) and row['jail_suspended_days'] != 0.0:
      jail_time -= pd.Timedelta(str(row['jail_suspended_days'] ) + " days")

  # if not pd.isna(row['jail_term_days']) and row['jail_term_days'] != 0.0:
  #   if jail_time is None:
  #     jail_time = pd.Timedelta(str(row['jail_term_days']) + " days")
  #   else:
  #     jail_time += pd.Timedelta(str(row['jail_term_days']) + " days")

  #   if not pd.isna(row['suspended_term_days']) and row['suspended_term_days'] != 0.0:
  #     jail_time -= pd.Timedelta(str(row['suspended_term_days']) + " days")

  if jail_time is None:
    # Assume jail time
    if 'Felony' in row['charge_class']:
      jail_time = three_half_year_delta
    else:
      jail_time = two_half_year_delta

  # statute code no present
  if row['statute_code'] not in df_brute_force['statute_code'].values:
    print(row['statute_code'])
    return "Ineligible"
  eligiblity_class = df_brute_force.loc[df_brute_force['statute_code']==row['statute_code']].charge_label.values[0]

  disposition_date = pd.to_datetime(row['disposition_date'], format="%Y-%m-%d")
  if eligiblity_class == '5M':
    eligible_date = disposition_date + jail_time + five_year_delta
  elif eligiblity_class == '7M':
    eligible_date = disposition_date + jail_time + seven_year_delta
  elif eligiblity_class == '15M':
    eligible_date = disposition_date + jail_time + fifteen_year_delta
  elif eligiblity_class == 'COIM':
    # Civil Offenses & Infractions
    eligible_date = disposition_date + jail_time + three_year_delta
  elif eligiblity_class == 'DCM':
    # Decriminalized Offenses
    eligible_date = disposition_date + jail_time
  elif eligiblity_class == '7F':
    eligible_date = disposition_date + jail_time + seven_year_delta
  elif eligiblity_class == '10F':
    eligible_date = disposition_date + jail_time + ten_year_delta
  else:
    return "Ineligible"
  if eligible_date <= current_date:
    return "Eligible"
  else:
    return "Ineligible"

In [ ]:
from joblib import parallel_backend, Parallel, delayed
results = Parallel(n_jobs=-1, backend="threading")(delayed(check_eligibility)(row) for _, row in df_guilty.iterrows())
df_guilty['eligibility'] = results
# df_guilty['eligibility'] = df_guilty.apply(check_eligibility, axis=1)

In [ ]:
df_guilty.to_csv('md_analysed.csv')

## Printing all numbers

In [2]:
# RUN THIS ONLY IF THE ABOVE CELLS AREN'T RUN
link_final = 'https://drive.google.com/file/d/1-1VwQp8DsRiAIi52stXKa1NIlfsXd2Pz/view?usp=drive_link'
# to get the id part of the file
id = link_final.split("/")[-2]

downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('md_analysed.csv')
df_guilty = pd.read_csv('md_analysed.csv', low_memory=False)

In [3]:
print("Races:", df.groupby('race').person_id.nunique())
print("\nGender:", df.sex.value_counts())
print("\nCharges in dataset:", df.shape)
print("\nNumber of people in sample:", df["person_id"].nunique())
print("\nNumber of convicted people in sample:", df_guilty["person_id"].nunique())
print("\nNumber of felony convicted people in sample:", df_guilty[df_guilty['charge_class'].isin(['Felony Circuit Court','Felony District Court'])]["person_id"].nunique())
print("\nNumber of misdemeanor convicted people in sample:", df_guilty[df_guilty['charge_class'].isin(['Misdemeanor'])]["person_id"].nunique())
print("\nNumber of people with felony charge in sample:", df[df['charge_class'].isin(['Felony Circuit Court','Felony District Court'])]["person_id"].nunique())
print("\nNumber of people eligible for relief:",df_guilty[(df_guilty['eligibility'] == 'Eligible')]["person_id"].nunique())

people_list = df.person_id.unique()
count_all_convictions_eligible = 0
count_partial_convictions_eligible = 0
count_fel_eligible = 0

for person in people_list:
    temp_df = df_guilty[df_guilty["person_id"] == person]
    eligibility_results = temp_df["eligibility"].unique()
    if "Eligible" in eligibility_results and "Ineligible" not in eligibility_results:
        count_all_convictions_eligible += 1
    elif "Eligible" in eligibility_results:
        count_partial_convictions_eligible += 1

    temp2_df = temp_df[temp_df["charge_class"].isin(['Felony Circuit Court','Felony District Court'])]
    eligibility_results = temp2_df["eligibility"].unique()
    if "Eligible" in eligibility_results:
        count_fel_eligible += 1

print("\nNumber of people with all convictions eligible:", count_all_convictions_eligible)
print("\nNumber of people with partial convictions eligible:", count_partial_convictions_eligible)
print("\nNumber of People with Felony Convictions with Clearable Felonies:", count_fel_eligible)

Races: race
AMERICAN INDIAN, ALASKA NATIVE                       67
ASIAN, NATIVE HAWAIIAN, OTHER PACIFIC ISLANDER      496
Asian                                               536
BLACK, AFRICAN AMERICAN                           26046
Black                                             15821
Indian                                               54
Native Hawaiian or Other Pacific Islander            28
Other                                               862
UNKNOWN, OTHER                                     1766
Unavailable                                         365
WHITE, CAUCASIAN, ASIATIC INDIAN, ARAB            10548
White                                             26580
Name: person_id, dtype: int64

Gender: Male       491863
M          301950
Female     230858
F          115528
U              31
Unknown         1
Name: sex, dtype: int64

Charges in dataset: (1164095, 75)

Number of people in sample: 75000

Number of convicted people in sample: 23941

Number of felony convicted pe

### Racial Numbers

In [ ]:
race_map = {
    "BLACK, AFRICAN AMERICAN":"BLACK",
"Black":"BLACK",
"WHITE, CAUCASIAN, ASIATIC INDIAN, ARAB":"WHITE",
"UNKNOWN, OTHER":"UNKNOWN/OTHER",
"Other":"UNKNOWN/OTHER",
"ASIAN, NATIVE HAWAIIAN, OTHER PACIFIC ISLANDER":"ASIAN",
"Unavailable":"UNKNOWN/OTHER",
"Asian":"ASIAN",
"Native Hawaiian or Other Pacific Islander":"NATIVE HAWAIIAN",
"AMERICAN INDIAN, ALASKA NATIVE":"AMERICAN INDIAN, ALASKA NATIVE",
"Indian":"AMERICAN INDIAN, ALASKA NATIVE",
"White":"WHITE"
}

df.race = df.race.replace(race_map)
df_guilty.race = df_guilty.race.replace(race_map)

In [ ]:
print("\nNumber of black people in sample:", df[df['race']=="BLACK"]['person_id'].nunique())
print("\nNumber of white people in sample:", df[df['race']=="WHITE"]['person_id'].nunique())
print("\nNumber of all people in sample:", df['person_id'].nunique())

print("\nNumber of black people convicted in sample:", df_guilty[df_guilty['race']=="BLACK"]['person_id'].nunique())
print("\nNumber of white people convicted in sample:", df_guilty[df_guilty['race']=="WHITE"]['person_id'].nunique())
print("\nNumber of all people convicted in sample:", df_guilty['person_id'].nunique())

print("\nNumber of black people that would have conviction after clearance of eligible in sample:", df_guilty[(df_guilty['race']=="BLACK") & (df_guilty['eligibility']=="Ineligible")]['person_id'].nunique())
print("\nNumber of white people that would have conviction after clearance of eligible in sample:", df_guilty[(df_guilty['race']=="WHITE") & (df_guilty['eligibility']=="Ineligible")]['person_id'].nunique())
print("\nNumber of all people that would have conviction after clearance of eligible in sample:", df_guilty[df_guilty['eligibility']=="Ineligible"]['person_id'].nunique())

print("\nNumber of black people felony convicted in sample:", df_guilty[(df_guilty['race']=="BLACK") & (df_guilty['charge_class'].str.contains('Felony'))]['person_id'].nunique())
print("\nNumber of white people felony convicted in sample:", df_guilty[(df_guilty['race']=="WHITE") & (df_guilty['charge_class'].str.contains('Felony'))]['person_id'].nunique())
print("\nNumber of all people felony convicted in sample:", df_guilty[df_guilty['charge_class'].str.contains('Felony')]['person_id'].nunique())

print("\nNumber of black people that would have felony conviction after clearance of eligible in sample:", df_guilty[(df_guilty['race']=="BLACK") & (df_guilty['eligibility']=="Ineligible") & (df_guilty['charge_class'].str.contains('Felony'))]['person_id'].nunique())
print("\nNumber of white people that would have felony conviction after clearance of eligible in sample:", df_guilty[(df_guilty['race']=="WHITE") & (df_guilty['eligibility']=="Ineligible") & (df_guilty['charge_class'].str.contains('Felony'))]['person_id'].nunique())
print("\nNumber of all people that would have felony conviction after clearance of eligible in sample:", df_guilty[(df_guilty['eligibility']=="Ineligible") & (df_guilty['charge_class'].str.contains('Felony'))]['person_id'].nunique())



Number of black people in sample: 37226

Number of white people in sample: 34497

Number of all people in sample: 75000

Number of black people convicted in sample: 12457

Number of white people convicted in sample: 11073

Number of all people convicted in sample: 23941

Number of black people that would have conviction after clearance of eligible in sample: 8182

Number of white people that would have conviction after clearance of eligible in sample: 6655

Number of all people that would have conviction after clearance of eligible in sample: 15108

Number of black people felony convicted in sample: 2537

Number of white people felony convicted in sample: 2405

Number of all people felony convicted in sample: 5113

Number of black people that would have felony conviction after clearance of eligible in sample: 1725

Number of white people that would have felony conviction after clearance of eligible in sample: 1586

Number of all people that would have felony conviction after clearance

### County-level Numbers

In [ ]:
link_final = 'https://drive.google.com/file/d/1L-VVJWIgvE_3TMs0s58Nab4PbUbm5oVa/view?usp=drive_link'
# to get the id part of the file
id = link_final.split("/")[-2]

downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('md_overview.csv')

df_moverview = pd.read_csv('md_overview.csv', low_memory=False)
df_moverview_orig = df.copy()

link_final = 'https://drive.google.com/file/d/1EW7SDOuE8NxsDLQnQAJBQmq4uy5YHCOh/view?usp=drive_link'
# to get the id part of the file
id = link_final.split("/")[-2]

downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('nmd_overview.csv')

df_noverview = pd.read_csv('nmd_overview.csv', low_memory=False)
df_noverview_orig = df.copy()

In [ ]:
df_noverview['location'] = df_noverview.court_system

In [ ]:
df_overview = pd.concat([df_moverview, df_noverview])

In [ ]:
df_overview

,case_number,location,court_system
0,2K00072424,Cecil,District Court For Cecil County - Criminal
1,5Y66116139,Kent,District Court For Kent County - Criminal
2,06K08037191,Carroll Circuit Court,Circuit Court For Carroll County - Criminal
3,2K00059810,Cecil,District Court For Cecil County - Criminal
4,3A00101440,Glen Burnie,District Court For Anne Arundel County - Criminal
...,...,...,...
2487250,6E00713040,DISTRICT COURT FOR PRINCE GEORGE'S COUNTY - CR...,DISTRICT COURT FOR PRINCE GEORGE'S COUNTY - CR...
2487251,6E00713292,DISTRICT COURT FOR PRINCE GEORGE'S COUNTY - CR...,DISTRICT COURT FOR PRINCE GEORGE'S COUNTY - CR...
2487252,6E00694679,DISTRICT COURT FOR PRINCE GEORGE'S COUNTY - CR...,DISTRICT COURT FOR PRINCE GEORGE'S COUNTY - CR...
2487253,6E00720033,DISTRICT COURT FOR PRINCE GEORGE'S COUNTY - CR...,DISTRICT COURT FOR PRINCE GEORGE'S COUNTY - CR...


In [ ]:
df = df.merge(df_overview, on='case_number', how='left')
df_guilty = df_guilty.merge(df_overview, on='case_number', how='left')

In [ ]:
df.location.unique()

array(["DISTRICT COURT FOR PRINCE GEORGE'S COUNTY - CRIMINAL SYSTEM",
       'Carroll', 'Snow Hill',
       'DISTRICT COURT FOR MONTGOMERY COUNTY - CRIMINAL SYSTEM',
       'DISTRICT COURT FOR BALTIMORE CITY - CRIMINAL SYSTEM',
       'Ocean City', 'Cecil', 'Annapolis', 'Essex', 'Towson',
       'Dorchester', 'Howard', 'Garrett', 'Charles', 'Queen Annes',
       'Harford', 'Glen Burnie', 'Carroll Circuit Court', 'Talbot',
       'Wicomico', 'Kent', 'Saint Marys Circuit Court', nan,
       'Anne Arundel Circuit Court', 'Catonsville',
       'Frederick Circuit Court', 'Harford Circuit Court', 'Allegany',
       'Frederick', 'Somerset', 'Washington',
       'Baltimore County Circuit Court', 'Queen Annes Circuit Court',
       'Cecil Circuit Court', 'Worcester Circuit Court', 'Saint Marys',
       'Charles Circuit Court', 'Calvert', 'Talbot Circuit Court',
       'Allegany Circuit Court', 'Dorchester Circuit Court',
       'Wicomico Circuit Court', 'Kent Circuit Court',
       'Howard Circ

In [ ]:
df_guilty.location.isna().sum()

3874

In [ ]:
# Group by city and take first non-null county
county_map = df.groupby('city')['location'].first()

# Forward fill missing values
county_map = county_map.fillna(method='ffill')

# Backward fill remaining missing values
county_map = county_map.fillna(method='bfill')

# Map the city to county mapping
df_guilty['location'] = df_guilty['city'].map(county_map)

# Group by city and take first non-null county
county_map = df_guilty.groupby('city')['location'].first()

# Forward fill missing values
county_map = county_map.fillna(method='ffill')

# Backward fill remaining missing values
county_map = county_map.fillna(method='bfill')

# Map the city to county mapping
df_guilty['location'] = df_guilty['city'].map(county_map)

In [ ]:
location_to_county = {
    "DISTRICT COURT FOR PRINCE GEORGE'S COUNTY - CRIMINAL SYSTEM": "Prince George's County",
  "Carroll": "Carroll County",
  "Snow Hill": "Worcester County",
  "DISTRICT COURT FOR MONTGOMERY COUNTY - CRIMINAL SYSTEM": "Montgomery County",
  "DISTRICT COURT FOR BALTIMORE CITY - CRIMINAL SYSTEM": "Baltimore City",
  "Ocean City": "Worcester County",
  "Cecil": "Cecil County",
  "Annapolis": "Anne Arundel County",
  "Essex": "Baltimore County",
  "Towson": "Baltimore County",
  "Dorchester": "Dorchester County",
  "Howard": "Howard County",
  "Garrett": "Garrett County",
  "Charles": "Charles County",
  "Queen Annes": "Queen Anne's County",
  "Harford": "Harford County",
  "Glen Burnie": "Anne Arundel County",
  "Carroll Circuit Court": "Carroll County",
  "Talbot": "Talbot County",
  "Wicomico": "Wicomico County",
  "Kent": "Kent County",
  "Saint Marys Circuit Court": "St. Mary's County",
  "Anne Arundel Circuit Court": "Anne Arundel County",
  "Catonsville": "Baltimore County",
  "Frederick Circuit Court": "Frederick County",
  "Harford Circuit Court": "Harford County",
  "Allegany": "Allegany County",
  "Frederick": "Frederick County",
  "Somerset": "Somerset County",
  "Washington": "Washington County",
  "Baltimore County Circuit Court": "Baltimore County",
  "Queen Annes Circuit Court": "Queen Anne's County",
  "Cecil Circuit Court": "Cecil County",
  "Worcester Circuit Court": "Worcester County",
  "Saint Marys": "St. Mary's County",
  "Charles Circuit Court": "Charles County",
  "Calvert": "Calvert County",
  "Talbot Circuit Court": "Talbot County",
  "Allegany Circuit Court": "Allegany County",
  "Dorchester Circuit Court": "Dorchester County",
  "Wicomico Circuit Court": "Wicomico County",
  "Kent Circuit Court": "Kent County",
  "Howard Circuit Court": "Howard County",
  "Somerset Circuit Court": "Somerset County",
  "Calvert Circuit Court": "Calvert County",
  "Caroline": "Caroline County",
  "Washington Circuit Court": "Washington County",
  "Prince Georges Circuit Court": "Prince George's County",
  "Garrett Circuit Court": "Garrett County",
  "Caroline Circuit Court": "Caroline County",
  "Silver Spring": "Montgomery County",
  "Montgomery Circuit Court": "Montgomery County",
  "Rockville": "Montgomery County",
  "Hyattsville": "Prince George's County"
}

df.location = df.location.replace(location_to_county)
df_guilty.location = df_guilty.location.replace(location_to_county)

In [ ]:
print("\nNumber of people in sample:", df.groupby("location")['person_id'].nunique().sort_values(ascending=False))
print("\nNumber of convicted people in sample:", df_guilty.groupby("location")['person_id'].nunique().sort_values(ascending=False))
print("\nNumber of felony convicted people in sample:", df_guilty[df_guilty['charge_class'].isin(['Felony Circuit Court','Felony District Court'])].groupby("location")['person_id'].nunique().sort_values(ascending=False))
print("\nNumber of misdemeanor convicted people in sample:", df_guilty[df_guilty['charge_class'].isin(['Misdemeanor'])].groupby("location")['person_id'].nunique().sort_values(ascending=False))
print("\nNumber of people with felony charge in sample:", df[df['charge_class'].isin(['Felony Circuit Court','Felony District Court'])].groupby("location")['person_id'].nunique().sort_values(ascending=False))
print("\nNumber of people eligible for relief:", df_guilty[(df_guilty['eligibility'] == 'Eligible')].groupby("location")['person_id'].nunique().sort_values(ascending=False))

df_counties = pd.DataFrame()
df_counties = df_counties.append(df.groupby("location")['person_id'].nunique().sort_values(ascending=False))
df_counties = df_counties.append(df_guilty.groupby("location")['person_id'].nunique().sort_values(ascending=False))
df_counties = df_counties.append(df_guilty[df_guilty['charge_class'].isin(['Felony Circuit Court','Felony District Court'])].groupby("location")['person_id'].nunique().sort_values(ascending=False))
df_counties = df_counties.append(df_guilty[df_guilty['charge_class'].isin(['Misdemeanor'])].groupby("location")['person_id'].nunique().sort_values(ascending=False))
df_counties = df_counties.append(df[df['charge_class'].isin(['Felony Circuit Court','Felony District Court'])].groupby("location")['person_id'].nunique().sort_values(ascending=False))
df_counties = df_counties.append(df_guilty[(df_guilty['eligibility'] == 'Eligible')].groupby("location")['person_id'].nunique().sort_values(ascending=False))

df_counties = df_counties.T
df_counties.columns = ['Number of people in sample', 'Number of convicted people in sample', 'Number of felony convicted people in sample', 'Number of misdemeanor convicted people in sample', 'Number of people with felony charge in sample', 'Number of people eligible for relief']
df_counties.to_csv('md_county.csv')

NameError: ignored

### Top-Ten Charges

In [ ]:
result = df.groupby('statute_code').agg(
    first_charge_description=('charge_description', 'first'),
    count=('charge_description', 'count')
)

# Sort the result by count in descending order
result = result.sort_values(by='count', ascending=False)

result[['first_charge_description','count']].head(10)

,first_charge_description,count
statute_code,,
CR.7.104,THEFT:LESS $500 VALUE,105832
CR.3.203,ASSAULT-SEC DEGREE,72212
CL,CON-LOTTERY POSS NUMBERS LIST,69009
CR.5.601.(a)(1),CDS:POSSESS-NOT MARIHUANA,65328
27.342,THEFT:LESS $500 VALUE,43565
CR.6.301,"MAL DEST PROP/VALU - $1,000",28994
CR.8.601.(a),FORGERY-PRIV DOCUMENTS,26475
CR.5.619.(c)(1),CDS:POSS PARAPHERNALIA,22968
CR.7.104.(g)(3),THEFT LESS THAN $100.00,21644


In [ ]:
result.reset_index()['count'].head(10)/df.shape[0]

0    0.090914
1    0.062033
2    0.059281
3    0.056119
4    0.037424
5    0.024907
6    0.022743
7    0.019730
8    0.018593
9    0.017993
Name: count, dtype: float64

In [ ]:
result = df_guilty[df_guilty['eligibility']=="Eligible"].groupby('statute_code').agg(
    first_charge_description=('charge_description', 'first'),
    count=('charge_description', 'count')
)

# Sort the result by count in descending order
result = result.sort_values(by='count', ascending=False)

result[['first_charge_description','count']].head(10)

,first_charge_description,count
statute_code,,
CR.7.104,THEFT: $500 PLUS VALUE,7069
CR.3.203,ASSAULT-SEC DEGREE,4413
27.342,THEFT:LESS $300 VALUE,4050
CL,ATT-THEFT: LESS $500 VALUE,3960
CR.5.619.(c)(1),CDS:POSS PARAPHERNALIA,2739
CR.10.201.(c)(2),DISORDERLY CONDUCT,1746
CR.6.301,MAL DEST PROP/VALU - $500,1478
27.287.(a),CDS: POSSESSION-MARIHUANA,1360
CR.7.104.(g)(3),THEFT LESS THAN $100.00,1313


In [ ]:
result.reset_index()['count'].head(10)/df_guilty[df_guilty['eligibility']=="Eligible"].shape[0]

0    0.146727
1    0.091598
2    0.084063
3    0.082195
4    0.056852
5    0.036241
6    0.030678
7    0.028229
8    0.027253
9    0.024928
Name: count, dtype: float64

In [ ]:
df_guilty[df_guilty['eligibility']=="Eligible"].shape[0]

48178